In [1]:
import xarray as xr
import requests
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Load .nc file
file_path="..\data\PBL_IMAGE_SSP2_categorial_land_cover_type_5min.nc"
dataset = xr.open_dataset(file_path)

# To see the structure of the file, print the dataset
print(dataset)

<xarray.Dataset>
Dimensions:    (latitude: 2160, longitude: 4320, time: 27)
Coordinates:
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * time       (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2100-01-01
Data variables:
    GLCT       (time, latitude, longitude) float64 ...
Attributes:
    title:        IMAGE 3.0, project: SSPs, scenario: SSP2, variable: GLCT
    conventions:  CF-1.6
    institution:  PBL Netherlands Environmental Assessment Agency (http://www...
    source:       IMAGE-LPJ; image25/MODEL/branches/imager1757#1833,interface...
    references:   http://www.pbl.nl/image;https://doi.org/10.1016/j.gloenvcha...
    history:       
    licence:      CC BY. The IMAGE-team would appreciate to be involved in pr...


In [4]:
dataset.data_vars["GLCT"]

<xarray.DataArray 'GLCT' (time: 27, latitude: 2160, longitude: 4320)>
[251942400 values with dtype=float64]
Coordinates:
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * time       (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2100-01-01
Attributes:
    qualifier:      agricultural land (1), extensive grassland (2), carbon pl...
    units:          -
    long_name:      categorial land cover type
    flag_values:    [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19...
    flag_meanings:  agricultural_land extensive_grassland carbon_plantation r...

In [4]:
land_cover_labels = {
    1: 'Agricultural Land', 2: 'Extensive Grassland', 3: 'Carbon Plantation', 4: 'Regrowth Forest Abandoning',
    5: 'Regrowth Forest Timber', 6: 'Biofuels', 7: 'Ice', 8: 'Tundra', 9: 'Wooded Tundra', 10: 'Boreal Forest',
    11: 'Cool Conifer Forest', 12: 'Temperate Mixed Forest', 13: 'Temperate Deciduous Forest', 14: 'Warm Mixed Forest',
    15: 'Grassland/Steppe', 16: 'Hot Desert', 17: 'Scrubland', 18: 'Savanna', 19: 'Tropical Woodland', 20: 'Tropical Forest'
}

In [5]:
# Specify the year as a string
year = '2050'

# Concatenate to form the full datetime string
datetime = f'{year}-01-01T00:00:00.000000000'

df = dataset.data_vars["GLCT"].sel(time=datetime).to_dataframe().reset_index()
gdf= gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [25]:
# check unique values of GLCT
lcts = df["GLCT"].unique()
lcts.sort()
print(lcts)
# len(df["GLCT"].unique())

[ 1.  2.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. nan]


In [6]:
# select rows that GLCT is not nan
gdf[gdf['GLCT'].notna()]

,latitude,longitude,time,GLCT,geometry
330088,83.625000,-32.625000,2050-01-01,7.0,POINT (-32.62500 83.62500)
334359,83.541667,-36.708333,2050-01-01,7.0,POINT (-36.70833 83.54167)
334360,83.541667,-36.625000,2050-01-01,7.0,POINT (-36.62500 83.54167)
334361,83.541667,-36.541667,2050-01-01,7.0,POINT (-36.54167 83.54167)
334362,83.541667,-36.458333,2050-01-01,7.0,POINT (-36.45833 83.54167)
...,...,...,...,...,...
7548381,-55.625000,-68.208333,2050-01-01,9.0,POINT (-68.20833 -55.62500)
7548382,-55.625000,-68.125000,2050-01-01,9.0,POINT (-68.12500 -55.62500)
7548383,-55.625000,-68.041667,2050-01-01,9.0,POINT (-68.04167 -55.62500)
7548384,-55.625000,-67.958333,2050-01-01,9.0,POINT (-67.95833 -55.62500)


In [7]:
# drop rows that GLCT is nan
gdf = gdf.dropna(subset=['GLCT'])

In [8]:
# change the geometry from point to polygon
half_side = 0.041667
gdf['geometry'] = gdf['geometry'].apply(lambda x: Polygon([[x.x-half_side, x.y-half_side], [x.x-half_side, x.y+half_side], [x.x+half_side, x.y+half_side], [x.x+half_side, x.y-half_side]]))

In [9]:
# get the input cropbox polygon
url = 'https://raw.githubusercontent.com/HotspotStoplight/HotspotStoplight/main/CropBoxes/CR_Crop3.geojson'
gdf2 = gpd.read_file(url)
area_polygon = gdf2.geometry[0]

In [10]:
area_gdf=gdf

In [11]:
# get the intersection of the landcover data and the area polygon
area_gdf['intersection'] = area_gdf.intersection(area_polygon)
area_gdf['area'] = area_gdf['intersection'].area

In [12]:
# drop rows that area is 0, which means there is no intersection with the input cropbox
area_gdf = area_gdf[area_gdf['area'] > 0]

# convert GLCT to int
area_gdf['GLCT'] = area_gdf['GLCT'].astype(int)

C:\Users\Jinze Wang\AppData\Roaming\Python\Python311\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [13]:
land_cover_areas = [0 for i in range(20)]
for i in range(20):
    land_cover_areas[i] = area_gdf[area_gdf['GLCT'] == i+1]['area'].sum()

total_area = sum(land_cover_areas)

In [14]:
landcover_fraction = []
for idx in range(1,21):
    curr_landcover_fraction = land_cover_areas[idx-1] / total_area
    landcover_fraction.append(curr_landcover_fraction)
    if land_cover_areas[idx-1] == 0:
        continue
    land_cover_name = land_cover_labels[idx]
    print(f'{land_cover_name}: {curr_landcover_fraction*100:.2f}%')

Agricultural Land: 48.38%
Regrowth Forest Timber: 12.93%
Savanna: 6.47%
Tropical Woodland: 8.34%
Tropical Forest: 23.88%


In [15]:
# now read the land polygon
land_file_path="..\data\land-polygons-complete-4326\land_polygons.shp"
land_gdf = gpd.read_file(land_file_path)


In [16]:
area_land_polygon_list=[]
remaining_area_polygon = area_polygon
# get the intersection of the area polygon and the land polygons
for idx in range(len(land_gdf.geometry)):
    intersection_polygon = land_gdf.geometry[idx].intersection(remaining_area_polygon)
    if intersection_polygon.is_empty:
        continue
    remaining_area_polygon = remaining_area_polygon - intersection_polygon
    # flat the multi-polygons to multiple polygons
    # if intersection_polygon.geom_type == 'MultiPolygon':
    #     for polygon in list(intersection_polygon.geoms):
    #         area_land_polygon_list.append(polygon)
    # else:
    #     area_land_polygon_list.append(intersection_polygon)
    area_land_polygon_list.append(intersection_polygon)


# get the area of the intersection
intersected_land_area = sum([x.area for x in area_land_polygon_list])
# calculate the area of each land cover type using the proportion we derived previously
output_dict_list=[]
for idx in range(1,21):
    land_cover_name = land_cover_labels[idx]
    land_cover_area = landcover_fraction[idx-1] * intersected_land_area
    curr_dict={"land_cover_name":land_cover_name, "land_cover_area":land_cover_area, "land_cover_proportion":landcover_fraction[idx-1]}
    output_dict_list.append(curr_dict)
    print(f'{land_cover_name}:  {landcover_fraction[idx-1]*100:.2f}% \t {land_cover_area:.2f}')


Agricultural Land:  48.38% 	 1.20
Extensive Grassland:  0.00% 	 0.00
Carbon Plantation:  0.00% 	 0.00
Regrowth Forest Abandoning:  0.00% 	 0.00
Regrowth Forest Timber:  12.93% 	 0.32
Biofuels:  0.00% 	 0.00
Ice:  0.00% 	 0.00
Tundra:  0.00% 	 0.00
Wooded Tundra:  0.00% 	 0.00
Boreal Forest:  0.00% 	 0.00
Cool Conifer Forest:  0.00% 	 0.00
Temperate Mixed Forest:  0.00% 	 0.00
Temperate Deciduous Forest:  0.00% 	 0.00
Warm Mixed Forest:  0.00% 	 0.00
Grassland/Steppe:  0.00% 	 0.00
Hot Desert:  0.00% 	 0.00
Scrubland:  0.00% 	 0.00
Savanna:  6.47% 	 0.16
Tropical Woodland:  8.34% 	 0.21
Tropical Forest:  23.88% 	 0.59


In [17]:
import pandas as pd

In [18]:
# write the output to a csv file
# convert the dictionary to a csv file
land_cover_demand_df = pd.DataFrame(output_dict_list)
# add the row key
land_cover_demand_df.to_csv('PBL_land_cover_demand.csv', index=False, header=True)

In [19]:
# def plot_polygon(polygon, ax, color):
#     x,y = polygon.exterior.xy
#     ax.fill(x, y, color=color, alpha=0.5)

In [20]:
# # show all polygons in the list in a plot
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots()
# plot_polygon(area_polygon, ax, 'blue')
# for idx in range(len(flat_area_land_polygon_list)):
#     plot_polygon(flat_area_land_polygon_list[idx], ax, 'red')
#     # area_land_polygon_list[idx].plot(ax=ax, color='red')
# plt.show()

In [21]:
# Note 
# the area of the input cropbox --- A0
# the area of the intersection of the input cropbox and the land cover data polygons(from pixels), without those with Nan land cover type --- A1
# the area of the intersection of the input cropbox and the land polygons A2

# A0>A1 since some of the pixels have Nan land cover type so they are not included in A1
# A0>A2 since some of the area in the input cropbox is covered by water, so they are not included in A2